# Machine Learning With Spark ML
In this lab assignment, you will complete a project by going through the following steps:
1. Get the data.
2. Discover the data to gain insights.
3. Prepare the data for Machine Learning algorithms.
4. Select a model and train it.
5. Fine-tune your model.
6. Present your solution.

As a dataset, we use the California Housing Prices dataset from the StatLib repository. This dataset was based on data from the 1990 California census. The dataset has the following columns
1. `longitude`: a measure of how far west a house is (a higher value is farther west)
2. `latitude`: a measure of how far north a house is (a higher value is farther north)
3. `housing_,median_age`: median age of a house within a block (a lower number is a newer building)
4. `total_rooms`: total number of rooms within a block
5. `total_bedrooms`: total number of bedrooms within a block
6. `population`: total number of people residing within a block
7. `households`: total number of households, a group of people residing within a home unit, for a block
8. `median_income`: median income for households within a block of houses
9. `median_house_value`: median house value for households within a block
10. `ocean_proximity`: location of the house w.r.t ocean/sea

---
# 1. Get the data
Let's start the lab by loading the dataset. The can find the dataset at `data/housing.csv`. To infer column types automatically, when you are reading the file, you need to set `inferSchema` to true. Moreover enable the `header` option to read the columns' name from the file.

In [59]:
// TODO: Replace <FILL IN> with appropriate code

val housing = spark.read.format("csv")
                .option("header", "true")
                .option("inferschema", "true")
                .load("data/housing.csv")

housing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 8 more fields]


---
# 2. Discover the data to gain insights
Now it is time to take a look at the data. In this step we are going to take a look at the data a few different ways:
* See the schema and dimension of the dataset
* Look at the data itself
* Statistical summary of the attributes
* Breakdown of the data by the categorical attribute variable
* Find the correlation among different attributes
* Make new attributes by combining existing attributes

## 2.1. Schema and dimension
Print the schema of the dataset

In [3]:
// TODO: Replace <FILL IN> with appropriate code

housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Print the number of records in the dataset.

In [4]:
// TODO: Replace <FILL IN> with appropriate code

housing.count()

res2: Long = 20640


## 2.2. Look at the data
Print the first five records of the dataset.

In [5]:
// TODO: Replace <FILL IN> with appropriate code

housing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

Print the number of records with population more than 10000.

In [6]:
// TODO: Replace <FILL IN> with appropriate code

housing.where("population > 10000").count()

res4: Long = 23


## 2.3. Statistical summary
Print a summary of the table statistics for the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. You can use the `describe` command.

In [7]:
// TODO: Replace <FILL IN> with appropriate code

housing.describe("housing_median_age", "total_rooms", "median_house_value", "population").show()

+-------+------------------+------------------+------------------+------------------+
|summary|housing_median_age|       total_rooms|median_house_value|        population|
+-------+------------------+------------------+------------------+------------------+
|  count|             20640|             20640|             20640|             20640|
|   mean|28.639486434108527|2635.7630813953488|206855.81690891474|1425.4767441860465|
| stddev| 12.58555761211163|2181.6152515827944|115395.61587441359|  1132.46212176534|
|    min|               1.0|               2.0|           14999.0|               3.0|
|    max|              52.0|           39320.0|          500001.0|           35682.0|
+-------+------------------+------------------+------------------+------------------+



Print the maximum age (`housing_median_age`), the minimum number of rooms (`total_rooms`), and the average of house values (`median_house_value`).

In [8]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.sql.functions._

housing.select(max("housing_median_age")).show()
housing.select(min("total_rooms")).show()
housing.select(avg("median_house_value")).show()

+-----------------------+
|max(housing_median_age)|
+-----------------------+
|                   52.0|
+-----------------------+

+----------------+
|min(total_rooms)|
+----------------+
|             2.0|
+----------------+

+-----------------------+
|avg(median_house_value)|
+-----------------------+
|     206855.81690891474|
+-----------------------+



import org.apache.spark.sql.functions._


## 2.4. Breakdown the data by categorical data
Print the number of houses in different areas (`ocean_proximity`), and sort them in descending order.

In [60]:
// TODO: Replace <FILL IN> with appropriate code

housing.groupBy("ocean_proximity")
        .sum("households")
        .sort(desc("sum(households)"))
        .show()

+---------------+---------------+
|ocean_proximity|sum(households)|
+---------------+---------------+
|      <1H OCEAN|      4730118.0|
|         INLAND|      3127759.0|
|     NEAR OCEAN|      1332308.0|
|       NEAR BAY|      1118931.0|
|         ISLAND|         1383.0|
+---------------+---------------+



Print the average value of the houses (`median_house_value`) in different areas (`ocean_proximity`), and call the new column `avg_value` when print it.

In [10]:
// TODO: Replace <FILL IN> with appropriate code

housing.groupBy("ocean_proximity")
        .agg(avg("median_house_value"))
        .withColumnRenamed("avg(median_house_value)", "avg_value")
        .show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



Rewrite the above question in SQL.

In [11]:
// TODO: Replace <FILL IN> with appropriate code

housing.createOrReplaceTempView("df")
spark.sql("SELECT ocean_proximity,avg(median_house_value) AS avg_value FROM df GROUP BY ocean_proximity").show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



## 2.5. Correlation among attributes
Print the correlation among the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. To do so, first you need to put these attributes into one vector. Then, compute the standard correlation coefficient (Pearson) between every pair of attributes in this new vector. To make a vector of these attributes, you can use the `VectorAssembler` Transformer.

In [13]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.VectorAssembler

val va = new VectorAssembler().setInputCols(Array("housing_median_age", "total_rooms", "median_house_value", "population")).setOutputCol("features")

val housingAttrs = va.transform(housing)

housingAttrs.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|            features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|[41.0,880.0,45260...|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|[21.0,7099.0,3585...|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|[52.0,1467.0,3521...|
|  -122.25|   37.85|              52.0|     12

import org.apache.spark.ml.feature.VectorAssembler
va: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_ca38ad97d999, handleInvalid=error, numInputCols=4
housingAttrs: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 9 more fields]


In [17]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row

val Row(coeff: Matrix) = Correlation.corr(housingAttrs, "features").head

println(s"The standard correlation coefficient:\n ${coeff}")

The standard correlation coefficient:
 1.0                   -0.3612622012223152  0.10562341249321029    -0.29624423977353675   
-0.3612622012223152   1.0                  0.13415311380656375    0.8571259728659744     
0.10562341249321029   0.13415311380656375  1.0                    -0.024649678888894997  
-0.29624423977353675  0.8571259728659744   -0.024649678888894997  1.0                    


import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row
coeff: org.apache.spark.ml.linalg.Matrix =
1.0                   -0.3612622012223152  0.10562341249321029    -0.29624423977353675
-0.3612622012223152   1.0                  0.13415311380656375    0.8571259728659744
0.10562341249321029   0.13415311380656375  1.0                    -0.024649678888894997
-0.29624423977353675  0.8571259728659744   -0.024649678888894997  1.0


## 2.6. Combine and make new attributes
Now, let's try out various attribute combinations. In the given dataset, the total number of rooms in a block is not very useful, if we don't know how many households there are. What we really want is the number of rooms per household. Similarly, the total number of bedrooms by itself is not very useful, and we want to compare it to the number of rooms. And the population per household seems like also an interesting attribute combination to look at. To do so, add the three new columns to the dataset as below. We will call the new dataset the `housingExtra`.
```
rooms_per_household = total_rooms / households
bedrooms_per_room = total_bedrooms / total_rooms
population_per_household = population / households
```

In [18]:
// TODO: Replace <FILL IN> with appropriate code

val housingCol1 = housing.withColumn("rooms_per_household", expr("total_rooms / households"))
val housingCol2 = housingCol1.withColumn("bedrooms_per_room", expr("total_bedrooms / total_rooms"))
val housingExtra = housingCol2.withColumn("population_per_household", expr("population / households"))

housingExtra.select("rooms_per_household", "bedrooms_per_room", "population_per_household").show(5)

+-------------------+-------------------+------------------------+
|rooms_per_household|  bedrooms_per_room|population_per_household|
+-------------------+-------------------+------------------------+
|  6.984126984126984|0.14659090909090908|      2.5555555555555554|
|  6.238137082601054|0.15579659106916466|       2.109841827768014|
|  8.288135593220339|0.12951601908657123|      2.8022598870056497|
| 5.8173515981735155|0.18445839874411302|       2.547945205479452|
|  6.281853281853282| 0.1720958819913952|      2.1814671814671813|
+-------------------+-------------------+------------------------+
only showing top 5 rows



housingCol1: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 9 more fields]
housingCol2: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 10 more fields]
housingExtra: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 11 more fields]


---
## 3. Prepare the data for Machine Learning algorithms
Before going through the Machine Learning steps, let's first rename the label column from `median_house_value` to `label`.

In [20]:
// TODO: Replace <FILL IN> with appropriate code

val renamedHousing = housingExtra.withColumnRenamed("median_house_value", "label")

renamedHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 11 more fields]


Now, we want to separate the numerical attributes from the categorical attribute (`ocean_proximity`) and keep their column names in two different lists. Moreover, sice we don't want to apply the same transformations to the predictors (features) and the label, we should remove the label attribute from the list of predictors. 

In [21]:
// label columns
val colLabel = "label"

// categorical columns
val colCat = "ocean_proximity"

// numerical columns
val colNum = renamedHousing.columns.filter(_ != colLabel).filter(_ != colCat)

colLabel: String = label
colCat: String = ocean_proximity
colNum: Array[String] = Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, rooms_per_household, bedrooms_per_room, population_per_household)


## 3.1. Prepare continuse attributes
### Data cleaning
Most Machine Learning algorithms cannot work with missing features, so we should take care of them. As a first step, let's find the columns with missing values in the numerical attributes. To do so, we can print the number of missing values of each continues attributes, listed in `colNum`.

In [25]:
// TODO: Replace <FILL IN> with appropriate code

for (c <- colNum) {
    val num_missing_values = renamedHousing.filter(renamedHousing(c).isNull).count()
    print(c + ": " + num_missing_values + "\n")
}

longitude: 0
latitude: 0
housing_median_age: 0
total_rooms: 0
total_bedrooms: 207
population: 0
households: 0
median_income: 0
rooms_per_household: 0
bedrooms_per_room: 207
population_per_household: 0


As we observerd above, the `total_bedrooms` and `bedrooms_per_room` attributes have some missing values. One way to take care of missing values is to use the `Imputer` Transformer, which completes missing values in a dataset, either using the mean or the median of the columns in which the missing values are located. To use it, you need to create an `Imputer` instance, specifying that you want to replace each attribute's missing values with the "median" of that attribute.

In [29]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.Imputer

val imputer = new Imputer()
                .setStrategy("median")
                .setInputCols(Array("total_bedrooms", "bedrooms_per_room"))
                .setOutputCols(Array("total_bedrooms", "bedrooms_per_room"))

val imputedHousing = imputer.fit(renamedHousing).transform(renamedHousing)

imputedHousing.select("total_bedrooms", "bedrooms_per_room").show(5)

+--------------+-------------------+
|total_bedrooms|  bedrooms_per_room|
+--------------+-------------------+
|         129.0|0.14659090909090908|
|        1106.0|0.15579659106916466|
|         190.0|0.12951601908657123|
|         235.0|0.18445839874411302|
|         280.0| 0.1720958819913952|
+--------------+-------------------+
only showing top 5 rows



import org.apache.spark.ml.feature.Imputer
imputer: org.apache.spark.ml.feature.Imputer = imputer_7a85223dec33
imputedHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 11 more fields]


### Scaling
One of the most important transformations you need to apply to your data is feature scaling. With few exceptions, Machine Learning algorithms don't perform well when the input numerical attributes have very different scales. This is the case for the housing data: the total number of rooms ranges from about 6 to 39,320, while the median incomes only range from 0 to 15. Note that scaling the label attribues is generally not required.

One way to get all attributes to have the same scale is to use standardization. In standardization, for each value, first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the variance so that the resulting distribution has unit variance. To do this, we can use the `StandardScaler` Estimator. To use `StandardScaler`, again we need to convert all the numerical attributes into a big vectore of features using `VectorAssembler`, and then call `StandardScaler` on that vactor.

In [33]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}

val va = new VectorAssembler().setInputCols(colNum).setOutputCol("features")
val featuredHousing = va.transform(imputedHousing)

val scaler = new StandardScaler().setInputCol("features").setOutputCol("scaled_features")
val scaledHousing = scaler.fit(featuredHousing).transform(featuredHousing)

scaledHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|            features|     scaled_features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|[-122.23,37.88,41...|[-61.007269596069...|
|  -122.22|   37.86|              21.0|     7099.0|        1

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}
va: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_1fda76f30077, handleInvalid=error, numInputCols=11
featuredHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 12 more fields]
scaler: org.apache.spark.ml.feature.StandardScaler = stdScal_a2b7575e5eef
scaledHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 13 more fields]


## 3.2. Prepare categorical attributes
After imputing and scaling the continuse attributes, we should take care of the categorical attributes. Let's first print the number of distict values of the categirical attribute `ocean_proximity`.

In [41]:
// TODO: Replace <FILL IN> with appropriate code

renamedHousing.select("ocean_proximity").distinct.count

res36: Long = 5


### String indexer
Most Machine Learning algorithms prefer to work with numbers. So let's convert the categorical attribute `ocean_proximity` to numbers. To do so, we can use the `StringIndexer` that encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), ordered by label frequencies, so the most frequent label gets index 0.

In [42]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.StringIndexer

val indexer = new StringIndexer().setInputCol("ocean_proximity").setOutputCol("ocean_proximity_index")
val idxHousing = indexer.fit(renamedHousing).transform(renamedHousing)

idxHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+---------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|ocean_proximity_index|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+---------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|                  3.0|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|358500.0|       NEAR BAY|  6.238137082

import org.apache.spark.ml.feature.StringIndexer
indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_7ebd29b60cdb
idxHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 12 more fields]


Now we can use this numerical data in any Machine Learning algorithm. You can look at the mapping that this encoder has learned using the `labels` method: "<1H OCEAN" is mapped to 0, "INLAND" is mapped to 1, etc.

In [43]:
indexer.fit(renamedHousing).labels

res38: Array[String] = Array(<1H OCEAN, INLAND, NEAR OCEAN, NEAR BAY, ISLAND)


### One-hot encoding
Now, convert the label indices built in the last step into one-hot vectors. To do this, you can take advantage of the `OneHotEncoderEstimator` Estimator.

In [48]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.OneHotEncoder

val encoder = new OneHotEncoder().setInputCol("ocean_proximity_index").setOutputCol("one-hot")
val ohHousing = encoder.fit(idxHousing).transform(idxHousing)

ohHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+---------------------+-------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|ocean_proximity_index|      one-hot|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+---------------------+-------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|                  3.0|(4,[3],[1.0])|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    113

import org.apache.spark.ml.feature.OneHotEncoder
encoder: org.apache.spark.ml.feature.OneHotEncoder = oneHotEncoder_7e3d4cdb218e
ohHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 13 more fields]


---
# 4. Pipeline
As you can see, there are many data transformation steps that need to be executed in the right order. For example, you called the `Imputer`, `VectorAssembler`, and `StandardScaler` from left to right. However, we can use the `Pipeline` class to define a sequence of Transformers/Estimators, and run them in order. A `Pipeline` is an `Estimator`, thus, after a Pipeline's `fit()` method runs, it produces a `PipelineModel`, which is a `Transformer`.

Now, let's create a pipeline called `numPipeline` to call the numerical transformers you built above (`imputer`, `va`, and `scaler`) in the right order from left to right, as well as a pipeline called `catPipeline` to call the categorical transformers (`indexer` and `encoder`). Then, put these two pipelines `numPipeline` and `catPipeline` into one pipeline.

In [49]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}

val numPipeline = new Pipeline().setStages(Array(imputer, va, scaler))
val catPipeline = new Pipeline().setStages(Array(indexer, encoder))
val pipeline = new Pipeline().setStages(Array(numPipeline, catPipeline))
val newHousing = pipeline.fit(renamedHousing).transform(renamedHousing)

newHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+---------------------+-------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|            features|     scaled_features|ocean_proximity_index|      one-hot|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+---------------------+-------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.555555555555

import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}
numPipeline: org.apache.spark.ml.Pipeline = pipeline_61952e7b5288
catPipeline: org.apache.spark.ml.Pipeline = pipeline_945ebc45dabd
pipeline: org.apache.spark.ml.Pipeline = pipeline_53a6895aad2b
newHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 15 more fields]


Now, use `VectorAssembler` to put all attributes of the final dataset `newHousing` into a big vector, and call the new column `features`.

In [60]:
// TODO: Replace <FILL IN> with appropriate code
val newHousingTemp = newHousing.drop("features")

val va2 = new VectorAssembler().setInputCols(Array("scaled_features", "one-hot")).setOutputCol("features")
val dataset = va2.transform(newHousingTemp).select("features", "label")

dataset.show(5)

+--------------------+--------+
|            features|   label|
+--------------------+--------+
|[-61.007269596069...|452600.0|
|[-61.002278409814...|358500.0|
|[-61.012260782324...|352100.0|
|[-61.017251968579...|341300.0|
|[-61.017251968579...|342200.0|
+--------------------+--------+
only showing top 5 rows



newHousingTemp: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 14 more fields]
va2: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_9dcea3f75719, handleInvalid=error, numInputCols=2
dataset: org.apache.spark.sql.DataFrame = [features: vector, label: double]


---
# 5. Make a model
Here we going to make four different regression models:
* Linear regression model
* Decission tree regression
* Random forest regression
* Gradient-booster forest regression

But, before giving the data to train a Machine Learning model, let's first split the data into training dataset (`trainSet`) with 80% of the whole data, and test dataset (`testSet`) with 20% of it.

In [61]:
// TODO: Replace <FILL IN> with appropriate code

val Array(trainSet, testSet) = dataset.randomSplit(Array(0.8, 0.2))

trainSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: double]
testSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: double]


## 5.1. Linear regression model
Now, train a Linear Regression model using the `LinearRegression` class. Then, print the coefficients and intercept of the model, as well as the summary of the model over the training set by calling the `summary` method.

In [65]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.LinearRegression

// train the model
val lr = new LinearRegression().setMaxIter(10)
val lrModel = lr.fit(trainSet)
val trainingSummary = lrModel.summary

println(s"Coefficients: ${lrModel.coefficients}, Intercept: ${lrModel.intercept}")
println(s"RMSE: ${trainingSummary.rootMeanSquaredError}")

Coefficients: [-56905.285786305,-58036.53412394746,13074.289945886523,10619.503699910787,-2455.211019954793,-54284.41663327252,49960.408458326296,75724.8563943924,6821.332070503445,15490.005338978444,1281.4885862653243,-122950.80773940812,-157326.21611410004,-118564.04729681858,-125730.52574537821], Intercept: -2351195.3903951664
RMSE: 67785.58640184777


import org.apache.spark.ml.regression.LinearRegression
lr: org.apache.spark.ml.regression.LinearRegression = linReg_4a243495325d
lrModel: org.apache.spark.ml.regression.LinearRegressionModel = LinearRegressionModel: uid=linReg_4a243495325d, numFeatures=15
trainingSummary: org.apache.spark.ml.regression.LinearRegressionTrainingSummary = org.apache.spark.ml.regression.LinearRegressionTrainingSummary@57db68bc


Now, use `RegressionEvaluator` to measure the root-mean-square-erroe (RMSE) of the model on the test dataset.

In [68]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.evaluation.RegressionEvaluator

// make predictions on the test data
val predictions = lrModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator()
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+-------+--------------------+
|        prediction|  label|            features|
+------------------+-------+--------------------+
|201702.33895226568|94600.0|[-62.065401082150...|
|206477.64791821223|50800.0|[-62.005506847089...|
|118605.89524147939|68400.0|[-61.995524474579...|
| 95064.87175383326|74600.0|[-61.985542102068...|
|161147.39460835326|70200.0|[-61.980550915813...|
+------------------+-------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 68673.84961679581


import org.apache.spark.ml.evaluation.RegressionEvaluator
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_f951736eabc0, metricName=rmse, throughOrigin=false
rmse: Double = 68673.84961679581


## 5.2. Decision tree regression
Repeat what you have done on Regression Model to build a Decision Tree model. Use the `DecisionTreeRegressor` to make a model and then measure its RMSE on the test dataset.

In [70]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val dt = new DecisionTreeRegressor()

// train the model
val dtModel = dt.fit(trainSet)

// make predictions on the test data
val predictions = dtModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator()
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+-------+--------------------+
|        prediction|  label|            features|
+------------------+-------+--------------------+
| 174819.2186746988|94600.0|[-62.065401082150...|
| 174819.2186746988|50800.0|[-62.005506847089...|
| 174819.2186746988|68400.0|[-61.995524474579...|
|         133833.75|74600.0|[-61.985542102068...|
|144406.23607427056|70200.0|[-61.980550915813...|
+------------------+-------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 67412.97635928713


import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
dt: org.apache.spark.ml.regression.DecisionTreeRegressor = dtr_1312649216ed
dtModel: org.apache.spark.ml.regression.DecisionTreeRegressionModel = DecisionTreeRegressionModel: uid=dtr_1312649216ed, depth=5, numNodes=63, numFeatures=15
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_f35a08500d5e, metricName=rmse, throughOrigin=false
rmse: Double = 67412.97635928713


## 5.3. Random forest regression
Let's try the test error on a Random Forest Model. Youcan use the `RandomForestRegressor` to make a Random Forest model.

In [81]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val rf = new RandomForestRegressor().setNumTrees(20).setMaxDepth(10)

// train the model
val rfModel = rf.fit(trainSet)

// make predictions on the test data
val predictions = rfModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator()
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+-------+--------------------+
|        prediction|  label|            features|
+------------------+-------+--------------------+
|144060.84444444443|94600.0|[-62.065401082150...|
|170644.12268907597|50800.0|[-62.005506847089...|
|113095.29008714075|68400.0|[-61.995524474579...|
| 93444.81569024145|74600.0|[-61.985542102068...|
| 95905.83317060437|70200.0|[-61.980550915813...|
+------------------+-------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 52621.170481700494


import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
rf: org.apache.spark.ml.regression.RandomForestRegressor = rfr_24520baadfeb
rfModel: org.apache.spark.ml.regression.RandomForestRegressionModel = RandomForestRegressionModel: uid=rfr_24520baadfeb, numTrees=20, numFeatures=15
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_26c85f42fcd6, metricName=rmse, throughOrigin=false
rmse: Double = 52621.170481700494


## 5.4. Gradient-boosted tree regression
Fianlly, we want to build a Gradient-boosted Tree Regression model and test the RMSE of the test data. Use the `GBTRegressor` to build the model.

In [84]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val gb = new GBTRegressor().setMaxIter(10)

// train the model
val gbModel = gb.fit(trainSet)

// make predictions on the test data
val predictions = gbModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator()
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+-------+--------------------+
|        prediction|  label|            features|
+------------------+-------+--------------------+
| 128727.5553495681|94600.0|[-62.065401082150...|
|122454.64640125002|50800.0|[-62.005506847089...|
|112960.59380530787|68400.0|[-61.995524474579...|
| 94260.08419473247|74600.0|[-61.985542102068...|
|109991.12897916697|70200.0|[-61.980550915813...|
+------------------+-------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 59019.63053754804


import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
gb: org.apache.spark.ml.regression.GBTRegressor = gbtr_805347d4769d
gbModel: org.apache.spark.ml.regression.GBTRegressionModel = GBTRegressionModel: uid=gbtr_805347d4769d, numTrees=10, numFeatures=15
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_72e102686a95, metricName=rmse, throughOrigin=false
rmse: Double = 59019.63053754804


---
# 6. Hyperparameter tuning
An important task in Machie Learning is model selection, or using data to find the best model or parameters for a given task. This is also called tuning. Tuning may be done for individual Estimators such as LinearRegression, or for entire Pipelines which include multiple algorithms, featurization, and other steps. Users can tune an entire Pipeline at once, rather than tuning each element in the Pipeline separately. MLlib supports model selection tools, such as `CrossValidator`. These tools require the following items:
* Estimator: algorithm or Pipeline to tune (`setEstimator`)
* Set of ParamMaps: parameters to choose from, sometimes called a "parameter grid" to search over (`setEstimatorParamMaps`)
* Evaluator: metric to measure how well a fitted Model does on held-out test data (`setEvaluator`)

`CrossValidator` begins by splitting the dataset into a set of folds, which are used as separate training and test datasets. For example with `k=3` folds, `CrossValidator` will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing. To evaluate a particular `ParamMap`, `CrossValidator` computes the average evaluation metric for the 3 Models produced by fitting the Estimator on the 3 different (training, test) dataset pairs. After identifying the best `ParamMap`, `CrossValidator` finally re-fits the Estimator using the best ParamMap and the entire dataset.

Below, use the `CrossValidator` to select the best Random Forest model. To do so, you need to define a grid of parameters. Let's say we want to do the search among the different number of trees (1, 5, and 10), and different tree depth (5, 10, and 15).

In [86]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.CrossValidator

val paramGrid = new ParamGridBuilder()
                .addGrid(rf.numTrees, Array(1, 5, 10))
                .addGrid(rf.maxDepth, Array(5, 10, 15))
                .build()

val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")

val cv = new CrossValidator()
        .setEstimator(rf)
        .setEvaluator(new RegressionEvaluator())
        .setEstimatorParamMaps(paramGrid)
        .setNumFolds(3)

val cvModel = cv.fit(trainSet)

val predictions = cvModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+-------+--------------------+
|        prediction|  label|            features|
+------------------+-------+--------------------+
|          150268.0|94600.0|[-62.065401082150...|
|          169940.0|50800.0|[-62.005506847089...|
|106789.44444444445|68400.0|[-61.995524474579...|
| 84968.20652173914|74600.0|[-61.985542102068...|
| 77823.69871794872|70200.0|[-61.980550915813...|
+------------------+-------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 52302.416066036145


import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.CrossValidator
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	rfr_24520baadfeb-maxDepth: 5,
	rfr_24520baadfeb-numTrees: 1
}, {
	rfr_24520baadfeb-maxDepth: 5,
	rfr_24520baadfeb-numTrees: 5
}, {
	rfr_24520baadfeb-maxDepth: 5,
	rfr_24520baadfeb-numTrees: 10
}, {
	rfr_24520baadfeb-maxDepth: 10,
	rfr_24520baadfeb-numTrees: 1
}, {
	rfr_24520baadfeb-maxDepth: 10,
	rfr_24520baadfeb-numTrees: 5
}, {
	rfr_24520baadfeb-maxDepth: 10,
	rfr_24520baadfeb-numTrees: 10
}, {
	rfr_24520baadfeb-maxDepth: 15,
	rfr_24520baadfeb-numTrees: 1
}, {
	rfr_24520baadfeb-maxDepth: 15,
	rfr_24520baadfeb-numTrees: 5
}, {
	rfr_24520baadfeb-maxDepth: 15,
	rfr_2452...


---
# 7. An End-to-End Classification Test
As the last step, you are given a dataset called `data/ccdefault.csv`. The dataset represents default of credit card clients. It has 30,000 cases and 24 different attributes. More details about the dataset is available at `data/ccdefault.txt`. In this task you should make three models, compare their results and conclude the ideal solution. Here are the suggested steps:
1. Load the data.
2. Carry out some exploratory analyses (e.g., how various features and the target variable are distributed).
3. Train a model to predict the target variable (risk of `default`).
  - Employ three different models (logistic regression, decision tree, and random forest).
  - Compare the models' performances (e.g., AUC).
  - Defend your choice of best model (e.g., what are the strength and weaknesses of each of these models?).
4. What more would you do with this data? Anything to help you devise a better solution?

## 7.1 Load Data

In [1]:
val credit = spark.read.format("csv")
                .option("header", "true")
                .option("inferschema", "true")
                .load("data/ccdefault.csv")

Intitializing Scala interpreter ...

Spark Web UI available at http://55ae930e6548:4040
SparkContext available as 'sc' (version = 3.2.0, master = local[*], app id = local-1637407685627)
SparkSession available as 'spark'


credit: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 23 more fields]


## 7.2 Exploratory Data Analysis

In [2]:
credit.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- LIMIT_BAL: integer (nullable = true)
 |-- SEX: integer (nullable = true)
 |-- EDUCATION: integer (nullable = true)
 |-- MARRIAGE: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- PAY_0: integer (nullable = true)
 |-- PAY_2: integer (nullable = true)
 |-- PAY_3: integer (nullable = true)
 |-- PAY_4: integer (nullable = true)
 |-- PAY_5: integer (nullable = true)
 |-- PAY_6: integer (nullable = true)
 |-- BILL_AMT1: integer (nullable = true)
 |-- BILL_AMT2: integer (nullable = true)
 |-- BILL_AMT3: integer (nullable = true)
 |-- BILL_AMT4: integer (nullable = true)
 |-- BILL_AMT5: integer (nullable = true)
 |-- BILL_AMT6: integer (nullable = true)
 |-- PAY_AMT1: integer (nullable = true)
 |-- PAY_AMT2: integer (nullable = true)
 |-- PAY_AMT3: integer (nullable = true)
 |-- PAY_AMT4: integer (nullable = true)
 |-- PAY_AMT5: integer (nullable = true)
 |-- PAY_AMT6: integer (nullable = true)
 |-- DEFAULT: integer (nullable = tru

In [3]:
credit.count()

res1: Long = 30000


In [4]:
credit.show(1)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|DEFAULT|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|      1|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+----

### Statistical summary

In [5]:
credit.describe("LIMIT_BAL").show()
credit.describe("AGE").show()
credit.describe("BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6").show() 
credit.describe("PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6").show()

+-------+------------------+
|summary|         LIMIT_BAL|
+-------+------------------+
|  count|             30000|
|   mean|167484.32266666667|
| stddev|129747.66156720246|
|    min|             10000|
|    max|           1000000|
+-------+------------------+

+-------+-----------------+
|summary|              AGE|
+-------+-----------------+
|  count|            30000|
|   mean|          35.4855|
| stddev|9.217904068090155|
|    min|               21|
|    max|               79|
+-------+-----------------+

+-------+-----------------+-----------------+-----------------+------------------+-----------------+----------------+
|summary|        BILL_AMT1|        BILL_AMT2|        BILL_AMT3|         BILL_AMT4|        BILL_AMT5|       BILL_AMT6|
+-------+-----------------+-----------------+-----------------+------------------+-----------------+----------------+
|  count|            30000|            30000|            30000|             30000|            30000|           30000|
|   mean|    

### Breakdown the data by categorical data

In [6]:
val cat_vars = Array("SEX", "EDUCATION", "MARRIAGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", 
                    "PAY_5", "PAY_6", "DEFAULT")

for (cat_var <- cat_vars){
    credit.groupBy(cat_var).count()
        .sort(desc("count"))
        .show()
}

+---+-----+
|SEX|count|
+---+-----+
|  2|18112|
|  1|11888|
+---+-----+

+---------+-----+
|EDUCATION|count|
+---------+-----+
|        2|14030|
|        1|10585|
|        3| 4917|
|        5|  280|
|        4|  123|
|        6|   51|
|        0|   14|
+---------+-----+

+--------+-----+
|MARRIAGE|count|
+--------+-----+
|       2|15964|
|       1|13659|
|       3|  323|
|       0|   54|
+--------+-----+

+-----+-----+
|PAY_0|count|
+-----+-----+
|    0|14737|
|   -1| 5686|
|    1| 3688|
|   -2| 2759|
|    2| 2667|
|    3|  322|
|    4|   76|
|    5|   26|
|    8|   19|
|    6|   11|
|    7|    9|
+-----+-----+

+-----+-----+
|PAY_2|count|
+-----+-----+
|    0|15730|
|   -1| 6050|
|    2| 3927|
|   -2| 3782|
|    3|  326|
|    4|   99|
|    1|   28|
|    5|   25|
|    7|   20|
|    6|   12|
|    8|    1|
+-----+-----+

+-----+-----+
|PAY_3|count|
+-----+-----+
|    0|15764|
|   -1| 5938|
|   -2| 4085|
|    2| 3819|
|    3|  240|
|    4|   76|
|    7|   27|
|    6|   23|
|    5|   21|
|

cat_vars: Array[String] = Array(SEX, EDUCATION, MARRIAGE, PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6, DEFAULT)


### Correlation among attributes

In [7]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row
import spark.implicits._

def correlation(attrs: Array[String], credit: org.apache.spark.sql.DataFrame) = {
    val va = new VectorAssembler().setInputCols(attrs).setOutputCol("comb")
    val creditAttrs = va.transform(credit)

    val Row(coeff: Matrix) = Correlation.corr(creditAttrs, "comb").head

    // convert Matrix to Dataframe to show all entries
    val cols = (0 until coeff.numCols).toSeq

    val df = coeff.transpose
      .colIter.toSeq
      .map(_.toArray)
      .toDF("arr")

    val df2 = cols.foldLeft(df)((df, i) => df.withColumn("_" + (i+1), $"arr"(i)))
      .drop("arr")
    
    df2.show()
}

import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row
import spark.implicits._
correlation: (attrs: Array[String], credit: org.apache.spark.sql.DataFrame)Unit


In [8]:
val BILL_AMT = Array("BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6")
val BILL_AMT_df = correlation(BILL_AMT, credit)

+------------------+------------------+------------------+------------------+------------------+------------------+
|                _1|                _2|                _3|                _4|                _5|                _6|
+------------------+------------------+------------------+------------------+------------------+------------------+
|               1.0|0.9514836727518373|0.8922785291271764|0.8602721890293002|0.8297786058330346|0.8026501885528918|
|0.9514836727518373|               1.0|0.9283262592714895|0.8924822912576753|0.8597783072714508|0.8315935591018394|
|0.8922785291271764|0.9283262592714895|               1.0| 0.923969456590977|0.8839096973619943| 0.853320090594039|
|0.8602721890293002|0.8924822912576753| 0.923969456590977|               1.0|0.9401344040880271|0.9009409547979023|
|0.8297786058330346|0.8597783072714508|0.8839096973619943|0.9401344040880271|               1.0|0.9461968070522467|
|0.8026501885528918|0.8315935591018394| 0.853320090594039|0.900940954797

BILL_AMT: Array[String] = Array(BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6)
BILL_AMT_df: Unit = ()


In [9]:
val PAY_AMT = Array("PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6")
val PAY_AMT_df = correlation(PAY_AMT, credit)

+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|                 _1|                 _2|                 _3|                 _4|                 _5|                 _6|
+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|                1.0| 0.2855755286868564|0.25219113895238066|0.19955793117066925| 0.1484592750153209|0.18573525544573963|
| 0.2855755286868564|                1.0|0.24477045029288216| 0.1801067436456434|0.18090775259419378| 0.1576339162723593|
|0.25219113895238066|0.24477045029288216|                1.0|0.21632509170083628|0.15921372030869177| 0.1627400332918404|
|0.19955793117066925| 0.1801067436456434|0.21632509170083628|                1.0|0.15183043582616404|0.15783391560138624|
| 0.1484592750153209|0.18090775259419378|0.15921372030869177|0.15183043582616404|                1.0|0.15489552525960376|
|0.18573525544573963| 0.

PAY_AMT: Array[String] = Array(PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6)
PAY_AMT_df: Unit = ()


In [10]:
val PAY = Array("PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6")
val PAY_df = correlation(PAY, credit)

+-------------------+------------------+------------------+------------------+------------------+-------------------+
|                 _1|                _2|                _3|                _4|                _5|                 _6|
+-------------------+------------------+------------------+------------------+------------------+-------------------+
|                1.0|0.6721643825485115|0.5742450926205569|0.5388406268711886|0.5094260636655171|0.47455308606419927|
| 0.6721643825485115|               1.0|0.7665516829340647|0.6620671310240177|0.6227802453771065| 0.5755008617794777|
| 0.5742450926205569|0.7665516829340647|               1.0|0.7773588733014175|0.6867745109946163| 0.6326835927185628|
| 0.5388406268711886|0.6620671310240177|0.7773588733014175|               1.0|0.8198353114868159|  0.716449481581117|
| 0.5094260636655171|0.6227802453771065|0.6867745109946163|0.8198353114868159|               1.0| 0.8169001604178695|
|0.47455308606419927|0.5755008617794777|0.63268359271856

PAY: Array[String] = Array(PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6)
PAY_df: Unit = ()


### Combine and make new attributes

In [11]:
// Average value among "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6"
val creditCol1 = credit.withColumn("AVG_PAY", 
                                   expr("PAY_0") + expr("PAY_2") + expr("PAY_3") + expr("PAY_4") + expr("PAY_5") + expr("PAY_6")/6)

// Average value among "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"
val creditCol2 = creditCol1.withColumn("AVG_PAY_AMT", 
                                       expr("PAY_AMT1") + expr("PAY_AMT2") + expr("PAY_AMT3") 
                                       + expr("PAY_AMT4") + expr("PAY_AMT5") + expr("PAY_AMT6")/6)

// Average value among "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6"
val creditExtra = creditCol2.withColumn("AVG_BILL_AMT", 
                                        expr("BILL_AMT1") + expr("BILL_AMT2") + expr("BILL_AMT3") 
                                        + expr("BILL_AMT4") + expr("BILL_AMT5") + expr("BILL_AMT6")/6)

creditExtra.select("AVG_PAY", "AVG_PAY_AMT", "AVG_BILL_AMT").show(5)

+-------------------+------------------+------------+
|            AVG_PAY|       AVG_PAY_AMT|AVG_BILL_AMT|
+-------------------+------------------+------------+
|-0.3333333333333333|             689.0|      7704.0|
| 1.3333333333333333|3333.3333333333335|     14359.5|
|                0.0| 6851.333333333333|     88695.5|
|                0.0| 7554.666666666667|    206711.5|
|               -2.0|58483.166666666664|     93396.5|
+-------------------+------------------+------------+
only showing top 5 rows



creditCol1: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 24 more fields]
creditCol2: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 25 more fields]
creditExtra: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 26 more fields]


## 7.3 Data Preparation

In [12]:
val renamedCredit = creditExtra.withColumnRenamed("DEFAULT", "label")

renamedCredit: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 26 more fields]


In [13]:
val colLabel = "label"

val colCat = Array("SEX", "EDUCATION", "MARRIAGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", 
                    "PAY_5", "PAY_6", "AVG_PAY")

val colNum = Array("LIMIT_BAL", "AGE", 
                   "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6",
                   "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6",
                  "AVG_PAY_AMT", "AVG_BILL_AMT")

colLabel: String = label
colCat: Array[String] = Array(SEX, EDUCATION, MARRIAGE, PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6, AVG_PAY)
colNum: Array[String] = Array(LIMIT_BAL, AGE, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6, AVG_PAY_AMT, AVG_BILL_AMT)


### Prepare continuse attributes

In [14]:
for (c <- colNum) {
    val num_missing_values = renamedCredit.filter(renamedCredit(c).isNull).count()
    print(c + ": " + num_missing_values + "\n")
}

LIMIT_BAL: 0
AGE: 0
BILL_AMT1: 0
BILL_AMT2: 0
BILL_AMT3: 0
BILL_AMT4: 0
BILL_AMT5: 0
BILL_AMT6: 0
PAY_AMT1: 0
PAY_AMT2: 0
PAY_AMT3: 0
PAY_AMT4: 0
PAY_AMT5: 0
PAY_AMT6: 0
AVG_PAY_AMT: 0
AVG_BILL_AMT: 0


In [15]:
import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}

val va = new VectorAssembler().setInputCols(colNum).setOutputCol("features")
val featuredCredit = va.transform(renamedCredit)

val scaler = new StandardScaler().setInputCol("features").setOutputCol("scaled_features")
val scaledCredit = scaler.fit(featuredCredit).transform(featuredCredit)

scaledCredit.show(1)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+-------------------+-----------+------------+--------------------+--------------------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|label|            AVG_PAY|AVG_PAY_AMT|AVG_BILL_AMT|            features|     scaled_features|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+-------------------+-----------+------------+--------------------+--------------------+
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|    

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}
va: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_c0fb54279451, handleInvalid=error, numInputCols=16
featuredCredit: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 27 more fields]
scaler: org.apache.spark.ml.feature.StandardScaler = stdScal_965195db947c
scaledCredit: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 28 more fields]


### Prepare categorical attributes

In [21]:
import org.apache.spark.ml.feature.StringIndexer

val indexerAvg = new StringIndexer().setInputCol("AVG_PAY").setOutputCol("AVG_PAY_index")
val idxCreditAvg = indexerAvg.fit(scaledCredit).transform(scaledCredit)

val indexer0 = new StringIndexer().setInputCol("PAY_0").setOutputCol("PAY_0_index")
val idxCredit0 = indexer0.fit(idxCreditAvg).transform(idxCreditAvg)

val indexer2 = new StringIndexer().setInputCol("PAY_2").setOutputCol("PAY_2_index")
val idxCredit2 = indexer2.fit(idxCredit0).transform(idxCredit0)

val indexer3 = new StringIndexer().setInputCol("PAY_3").setOutputCol("PAY_3_index")
val idxCredit3 = indexer3.fit(idxCredit2).transform(idxCredit2)

val indexer4 = new StringIndexer().setInputCol("PAY_4").setOutputCol("PAY_4_index")
val idxCredit4 = indexer4.fit(idxCredit3).transform(idxCredit3)

val indexer5 = new StringIndexer().setInputCol("PAY_5").setOutputCol("PAY_5_index")
val idxCredit5 = indexer5.fit(idxCredit4).transform(idxCredit4)

val indexer6 = new StringIndexer().setInputCol("PAY_6").setOutputCol("PAY_6_index")
val idxCredit = indexer6.fit(idxCredit5).transform(idxCredit5)

idxCredit.show(1)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+-------------------+-----------+------------+--------------------+--------------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|label|            AVG_PAY|AVG_PAY_AMT|AVG_BILL_AMT|            features|     scaled_features|AVG_PAY_index|PAY_0_index|PAY_2_index|PAY_3_index|PAY_4_index|PAY_5_index|PAY_6_index|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+-------------------+-----------+------------+-------------

import org.apache.spark.ml.feature.StringIndexer
indexerAvg: org.apache.spark.ml.feature.StringIndexer = strIdx_72ff651432c8
idxCreditAvg: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 29 more fields]
indexer0: org.apache.spark.ml.feature.StringIndexer = strIdx_9a4d1ef980aa
idxCredit0: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 30 more fields]
indexer2: org.apache.spark.ml.feature.StringIndexer = strIdx_5a764739b046
idxCredit2: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 31 more fields]
indexer3: org.apache.spark.ml.feature.StringIndexer = strIdx_f329f92ed971
idxCredit3: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 32 more fields]
indexer4: org.apache.spark.ml.feature.StringIndexer = strIdx_e32486520ea9
idxCredit4: or...


In [22]:
import org.apache.spark.ml.feature.OneHotEncoder

val catArray = Array("SEX", "EDUCATION", "MARRIAGE", 
                     "PAY_0_index", "PAY_2_index", "PAY_3_index", "PAY_4_index", 
                    "PAY_5_index", "PAY_6_index", "AVG_PAY_index")

val encoderSEX = new OneHotEncoder()
            .setInputCol("SEX")
            .setOutputCol("SEX_oh")
val ohCreditSEX = encoderSEX.fit(idxCredit).transform(idxCredit)

val encoderEDUCATION = new OneHotEncoder()
            .setInputCol("EDUCATION")
            .setOutputCol("EDUCATION_oh")
val ohCreditEDUCATION = encoderEDUCATION.fit(ohCreditSEX).transform(ohCreditSEX)

val encoderMARRIAGE = new OneHotEncoder()
            .setInputCol("MARRIAGE")
            .setOutputCol("MARRIAGE_oh")
val ohCreditMARRIAGE = encoderMARRIAGE.fit(ohCreditEDUCATION).transform(ohCreditEDUCATION)

val encoder0 = new OneHotEncoder()
            .setInputCol("PAY_0_index")
            .setOutputCol("PAY_0_index_oh")
val ohCredit0 = encoder0.fit(ohCreditMARRIAGE).transform(ohCreditMARRIAGE)

val encoder2 = new OneHotEncoder()
            .setInputCol("PAY_2_index")
            .setOutputCol("PAY_2_index_oh")
val ohCredit2 = encoder2.fit(ohCredit0).transform(ohCredit0)

val encoder3 = new OneHotEncoder()
            .setInputCol("PAY_3_index")
            .setOutputCol("PAY_3_index_oh")
val ohCredit3 = encoder3.fit(ohCredit2).transform(ohCredit2)

val encoder4 = new OneHotEncoder()
            .setInputCol("PAY_4_index")
            .setOutputCol("PAY_4_index_oh")
val ohCredit4 = encoder4.fit(ohCredit3).transform(ohCredit3)

val encoder5 = new OneHotEncoder()
            .setInputCol("PAY_5_index")
            .setOutputCol("PAY_5_index_oh")
val ohCredit5 = encoder5.fit(ohCredit4).transform(ohCredit4)

val encoder6 = new OneHotEncoder()
            .setInputCol("PAY_6_index")
            .setOutputCol("PAY_6_index_oh")
val ohCredit6 = encoder6.fit(ohCredit5).transform(ohCredit5)

val encoderAVG = new OneHotEncoder()
            .setInputCol("AVG_PAY_index")
            .setOutputCol("AVG_PAY_index_oh")
val ohCredit = encoderAVG.fit(ohCredit6).transform(ohCredit6)

ohCredit.show(1)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+-------------------+-----------+------------+--------------------+--------------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+---------+-------------+-------------+--------------+--------------+--------------+--------------+--------------+--------------+----------------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|label|            AVG_PAY|AVG_PAY_AMT|AVG_BILL_AMT|            features|     scaled_features|AVG_PAY_index|PAY_0_index|PAY_2_index|PAY_3_index|PAY_4_index|PAY_5_index|PAY_6_index|   SEX_oh| EDUCATION_oh|  MARRIAGE_oh|PAY_0_index_oh|PAY_2_index_oh|PAY_3_index_oh|PAY_4_index_oh|PAY_5_index_oh

import org.apache.spark.ml.feature.OneHotEncoder
catArray: Array[String] = Array(SEX, EDUCATION, MARRIAGE, PAY_0_index, PAY_2_index, PAY_3_index, PAY_4_index, PAY_5_index, PAY_6_index, AVG_PAY_index)
encoderSEX: org.apache.spark.ml.feature.OneHotEncoder = oneHotEncoder_e389590fdb78
ohCreditSEX: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 36 more fields]
encoderEDUCATION: org.apache.spark.ml.feature.OneHotEncoder = oneHotEncoder_6691aa934dd7
ohCreditEDUCATION: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 37 more fields]
encoderMARRIAGE: org.apache.spark.ml.feature.OneHotEncoder = oneHotEncoder_461ba83ae51e
ohCreditMARRIAGE: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 38 more fields]
encoder0: org.apache.spark.ml.feature.OneHotEncoder =...


### Final sets

In [23]:
ohCredit.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- LIMIT_BAL: integer (nullable = true)
 |-- SEX: integer (nullable = true)
 |-- EDUCATION: integer (nullable = true)
 |-- MARRIAGE: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- PAY_0: integer (nullable = true)
 |-- PAY_2: integer (nullable = true)
 |-- PAY_3: integer (nullable = true)
 |-- PAY_4: integer (nullable = true)
 |-- PAY_5: integer (nullable = true)
 |-- PAY_6: integer (nullable = true)
 |-- BILL_AMT1: integer (nullable = true)
 |-- BILL_AMT2: integer (nullable = true)
 |-- BILL_AMT3: integer (nullable = true)
 |-- BILL_AMT4: integer (nullable = true)
 |-- BILL_AMT5: integer (nullable = true)
 |-- BILL_AMT6: integer (nullable = true)
 |-- PAY_AMT1: integer (nullable = true)
 |-- PAY_AMT2: integer (nullable = true)
 |-- PAY_AMT3: integer (nullable = true)
 |-- PAY_AMT4: integer (nullable = true)
 |-- PAY_AMT5: integer (nullable = true)
 |-- PAY_AMT6: integer (nullable = true)
 |-- label: integer (nullable = true)

In [32]:
val temp = ohCredit.drop("features")

val va = new VectorAssembler()
        .setInputCols(Array("scaled_features", "SEX_oh", "EDUCATION_oh", "MARRIAGE_oh",
                           "AVG_PAY_index_oh"))
        .setOutputCol("features")

val dataset = va.transform(temp).select("features", "label")

dataset.show(1, false)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                                                                          |label|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|(184,[0,1,2,3,4,9,14,15,20,25,57],[0.15414535998894324,2.603628744963987,0.053139869207970765,0.0435834725779124,0.009935199510232218,0.029903384202815683,0.012651887788133186,0.023067198849194535,1.0,1.0,1.0])|1    |
+---------------------------------------------------------------------------------------------------------------------------

temp: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 44 more fields]
va: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_cfb959d20820, handleInvalid=error, numInputCols=5
dataset: org.apache.spark.sql.DataFrame = [features: vector, label: int]


## 7.4 Make a Model

In [33]:
val Array(trainSet, testSet) = dataset.randomSplit(Array(0.8, 0.2))

trainSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: int]
testSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: int]


### Logistic Regression

In [47]:
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

// train the model
val lr = new LogisticRegression().setMaxIter(10)
val lrModel = lr.fit(trainSet)

// Summarize the model over the training set and print out some metrics
val trainSummary = lrModel.summary
println(s"Train accuracy: ${trainSummary.accuracy}")
println(s"Train weightedPrecision: ${trainSummary.weightedPrecision}")
println(s"Train weightedRecall: ${trainSummary.weightedRecall}")
println(s"Train weightedFMeasure: ${trainSummary.weightedFMeasure}")

val predictions = lrModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// AUC
val evaluator = new BinaryClassificationEvaluator()
  .setLabelCol("label")
  .setRawPredictionCol("rawPrediction")
  .setMetricName("areaUnderROC")
val auc = evaluator.evaluate(predictions)
print(s"Test AUC: ${auc}")

Train accuracy: 0.8078141939788175
Train weightedPrecision: 0.787382305340163
Train weightedRecall: 0.8078141939788175
Train weightedFMeasure: 0.7792924074330573
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(184,[0,1,2,3,4,5...|
|       0.0|    0|(184,[0,1,2,3,4,5...|
|       0.0|    0|(184,[0,1,2,3,4,5...|
|       0.0|    0|(184,[0,1,2,3,4,5...|
|       0.0|    0|(184,[0,1,2,3,4,5...|
+----------+-----+--------------------+
only showing top 5 rows

Test AUC: 0.7665823825503356

import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
lr: org.apache.spark.ml.classification.LogisticRegression = logreg_1efc16cdba3d
lrModel: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid=logreg_1efc16cdba3d, numClasses=2, numFeatures=184
trainSummary: org.apache.spark.ml.classification.LogisticRegressionTrainingSummary = org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@2ac543de
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: int ... 3 more fields]
evaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = BinaryClassificationEvaluator: uid=binEval_f8ed0faf5089, metricName=areaUnderROC, numBins=1000
a...


### Decision Tree

In [53]:
import org.apache.spark.ml.classification.DecisionTreeClassifier

val dt = new DecisionTreeClassifier().setMaxDepth(30)

// train the model
val dtModel = dt.fit(trainSet)

// make predictions on the test data
val predictions = dtModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new BinaryClassificationEvaluator()
  .setLabelCol("label")
  .setRawPredictionCol("rawPrediction")
  .setMetricName("areaUnderROC")
val auc = evaluator.evaluate(predictions)
print(s"Test AUC: ${auc}")

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(184,[0,1,2,3,4,5...|
|       0.0|    0|(184,[0,1,2,3,4,5...|
|       0.0|    0|(184,[0,1,2,3,4,5...|
|       0.0|    0|(184,[0,1,2,3,4,5...|
|       1.0|    0|(184,[0,1,2,3,4,5...|
+----------+-----+--------------------+
only showing top 5 rows

Test AUC: 0.6162842281879195

import org.apache.spark.ml.classification.DecisionTreeClassifier
dt: org.apache.spark.ml.classification.DecisionTreeClassifier = dtc_57d0950244fb
dtModel: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel: uid=dtc_57d0950244fb, depth=30, numNodes=8203, numClasses=2, numFeatures=184
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: int ... 3 more fields]
evaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = BinaryClassificationEvaluator: uid=binEval_62b86dec7bdf, metricName=areaUnderROC, numBins=1000
auc: Double = 0.6162842281879195


### Random Forest

In [57]:
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}

val rf = new RandomForestClassifier().setNumTrees(100).setMaxDepth(10)

// train the model
val rfModel = rf.fit(trainSet)

// make predictions on the test data
val predictions = rfModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new BinaryClassificationEvaluator()
  .setLabelCol("label")
  .setRawPredictionCol("rawPrediction")
  .setMetricName("areaUnderROC")
val auc = evaluator.evaluate(predictions)
print(s"Test AUC: ${auc}")

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(184,[0,1,2,3,4,5...|
|       0.0|    0|(184,[0,1,2,3,4,5...|
|       0.0|    0|(184,[0,1,2,3,4,5...|
|       0.0|    0|(184,[0,1,2,3,4,5...|
|       0.0|    0|(184,[0,1,2,3,4,5...|
+----------+-----+--------------------+
only showing top 5 rows

Test AUC: 0.7763329697986566

import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_010546f4357a
rfModel: org.apache.spark.ml.classification.RandomForestClassificationModel = RandomForestClassificationModel: uid=rfc_010546f4357a, numTrees=100, numClasses=2, numFeatures=184
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: int ... 3 more fields]
evaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = BinaryClassificationEvaluator: uid=binEval_511b8894082e, metricName=areaUnderROC, numBins=1000
auc: Double = 0.7763329697986566


## 7.5 Analysis

### Defend your choice of best model (e.g., what are the strength and weaknesses of each of these models?

The test AUC values of three models are:<br />

    Logistic regression: 0.7665823825503356<br />
    Decision Tree: 0.6162842281879195<br />
    Random Forest: 0.7763329697986566

In this case, the best performed model is the random forest. The logistic regression model also works good, only 1% less than the random forest model.

The logistic regression model is usually applied on binary classification problem. While applying it on multi-class problem, we need to use softmax function instead of the logistic function. The logistic regression model is straight forward as we can get the coefficient of each feature directly.

The decision tree model is simple and explanable. However, it does not perform well in our case. We tuned the max depth parameter of the tree model, and find that in some extend, increasing the max-depth will improve the performance.

The random forest model is a resemble method based on decision tree. It grows many different dicision trees and let those trees vote for the final classification result. Therefore, it can always have a reasonable performance and avoid overfitting.

### What more would you do with this data? Anything to help you devise a better solution?

In the above procedure, we splited the dataset into training set and test set at once and chose hyperparameters arbitrarily by experience. However, we can apply cross-validation with grids setting to tune the hyperparameters and probably can get robuster results.

We can also try other evaluators to compare different models, for example, the accuracy, precision, recall, F-1 score and confusion matirx. These evaluation quantities will give us a more comprehensive model comparison。

### References:
[1] https://spark.apache.org/docs/latest/ml-classification-regression.html#classification-and-regression

[2] https://zhuanlan.zhihu.com/p/110664865